In [3]:
import dash
import dash_bootstrap_components as dbc
from dash import Input, Output, State, html, dcc
from pathlib import Path
import pandas as pd 
import numpy as np
import pickle
import json
from area import area
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

In [4]:
listingRaw = pd.read_csv("http://data.insideairbnb.com/france/ile-de-france/paris/2023-12-12/data/listings.csv.gz", 
                         low_memory=False)
calendarRaw = pd.read_csv("http://data.insideairbnb.com/france/ile-de-france/paris/2023-12-12/data/calendar.csv.gz", 
                          low_memory=False)
quartierGeo = json.load(open(file="neighbourhoods.geojson", mode="r", encoding='utf-8'))

In [5]:
listing = listingRaw.copy()
calendar = calendarRaw.copy()

In [6]:
prix_m2_arr="""1er arrondissement : 13.445 €/m2
2eme arrondissement : 12.570 €/m2
3eme arrondissement : 12.982 €/m2
4eme arrondissement : 13.928 €/m2
5eme arrondissement : 13.186 €/m2
6eme arrondissement : 15.367 €/m2
7eme arrondissement : 14.827 €/m2
8eme arrondissement : 12.510 €/m2
9eme arrondissement : 11.872 €/m2
10eme arrondissement : 11.065 €/m2
11eme arrondissement : 11.305 €/m2
12eme arrondissement : 10.355 €/m2
13eme arrondissement : 9.916 €/m2
14eme arrondissement : 10.805 €/m2
15eme arrondissement : 10.976 €/m2
16eme arrondissement : 12.086 €/m2
17eme arrondissement : 11.767 €/m2
18eme arrondissement : 10.855 €/m2
19eme arrondissement : 9.475 €/m2
20eme arrondissement : 9.874 €/m2""".splitlines()

nomToNumero = {
    "Louvre":"1er",
    "Bourse":"2eme",
    "Temple":"3eme",
    "Hôtel-de-Ville":"4eme",
    "Panthéon":"5eme",
    "Luxembourg":"6eme",
    "Palais-Bourbon":"7eme",
    "Élysée":"8eme",
    "Opéra":"9eme",
    "Entrepôt":"10eme",
    "Popincourt":"11eme",
    "Reuilly":"12eme",
    "Gobelins":"13eme",
    "Observatoire":"14eme",
    "Vaugirard":"15eme",
    "Passy":"16eme",
    "Batignolles-Monceau":"17eme",
    "Buttes-Montmartre":"18eme",
    "Buttes-Chaumont":"19eme",
    "Ménilmontant":"20eme"
}

numeroToNom = {v: k for k, v in nomToNumero.items()}

In [7]:
calendar = calendar[calendar.price.notnull()]
listing = listing[listing.price.notnull()]
listing['bedrooms'] = listing['bedrooms'].fillna(0)
listing["accommodates"] = listing["accommodates"].replace(0, np.nan)
listing["price"] = listing["price"].apply(lambda x: int(float(x.replace("$", "").replace(",", ""))))

keys = [numeroToNom[a] for a in [a[0:a.find(" ")] for a in prix_m2_arr]]
values = [int(a[a.find(":")+2:a.find("€")-1].replace(".","")) for a in prix_m2_arr]
pxmc = pd.DataFrame({"neighbourhood" : keys, "PrixMcarré": values})
pxmc["Arrondissement"] = pxmc["neighbourhood"].apply(lambda x: nomToNumero[x])

neighbourhoodArea = dict()
for cartier in quartierGeo["features"]:
    neighbourhoodArea[cartier["properties"]["neighbourhood"]] = round(area(cartier["geometry"]))
neighbourhoodArea["Reuilly"] += -9950000 #J'enlève la superficie du bois de Vincenne qui fait partie du 12eme
#Nombre de logement par quartier:
densite = pd.DataFrame(listing.groupby(["neighbourhood_cleansed"])["neighbourhood_cleansed"].count().reset_index(name="#logements"))
densite["superficie"] = densite["neighbourhood_cleansed"].apply(lambda x: neighbourhoodArea[x])
#nombre de logement par hectares par quartier
densite["nb/hectares"] = round(densite["#logements"]/(densite["superficie"]/10000), 2)
densite = densite.rename(columns={"neighbourhood_cleansed" : 'neighbourhood'})
densite["Arrondissement"] = densite["neighbourhood"].apply(lambda x: nomToNumero[x])

prix = listing.loc[listing["accommodates"]==2].groupby(["neighbourhood_cleansed"])["price"].mean().reset_index(name="Prix moyen")
prix["Prix moyen"] = round(prix["Prix moyen"],2)
prix = prix.rename(columns={"neighbourhood_cleansed":"neighbourhood"})
prix["Arrondissement"] = prix["neighbourhood"].apply(lambda x: nomToNumero[x])

score = listing.groupby("neighbourhood_cleansed").mean("review_scores_rating")["review_scores_rating"].reset_index(name="Evaluation")
score["Arrondissement"] = score["neighbourhood_cleansed"].apply(lambda x: nomToNumero[x])
score["Evaluation"] = round(score["Evaluation"],2)

value = listing.groupby("neighbourhood_cleansed").mean("review_scores_value")["review_scores_value"].reset_index(name="Evaluation")
value["Arrondissement"] = value["neighbourhood_cleansed"].apply(lambda x: nomToNumero[x])
value["Evaluation"] = round(value["Evaluation"],2)

listing.loc[listing["bedrooms"]==0, "bedrooms"] = 0.999
listing["prixParChambre"] = listing["price"]/listing["bedrooms"]
listing.loc[listing["bedrooms"]==0.999, "bedrooms"] = 0
pPC = listing.groupby("bedrooms")["prixParChambre"].mean().reset_index(name="prixParChambre").rename(columns={"bedrooms":"chambres"})
pPC["prixParChambre"] = pPC["prixParChambre"].apply(lambda x : round(x, 2))
pPC["chambres"] = pPC["chambres"].apply(lambda x : round(x))
pPC["Prix/chambre"] = pPC['prixParChambre'].apply(lambda x: f"{round(x)}€")
pPC = pPC.loc[pPC["chambres"] <= 4]


Logements = listing.groupby(["room_type"])["room_type"].count()\
            .reset_index(name="#logements")\
            .sort_values(by=['#logements'],ascending=False)
Logements["room_type"] = ["Logement entier", "Chambre perso", "Chambre d'hotel","Dortoir"]

repartition = listing[['price', 'neighbourhood_cleansed', 'bedrooms']]

datas = [pxmc,densite,prix,score,value,pPC,Logements,repartition]

with open(r"MyDashApp/src/data.pickle", 'wb') as f:
    pickle.dump(datas, f)

In [55]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])
#HERE = Path(__file__).parent

def cartePrixAuMetreCarre():
    keys = [numeroToNom[a] for a in [a[0:a.find(" ")] for a in prix_m2_arr]]
    values = [int(a[a.find(":")+2:a.find("€")-1].replace(".","")) for a in prix_m2_arr]
    mapData = pd.DataFrame({"neighbourhood" : keys, "PrixMcarré": values})
    mapData["Arrondissement"] = mapData["neighbourhood"].apply(lambda x: nomToNumero[x])
    fig = px.choropleth_mapbox(mapData, geojson=quartierGeo, color='PrixMcarré',
                locations='neighbourhood', featureidkey="properties.neighbourhood",
                mapbox_style="carto-positron", center={"lat":48.86, "lon": 2.35}, zoom=11,
                opacity=0.7, color_continuous_scale='purples', 
                hover_data={
                    "neighbourhood":False,
                    "Arrondissement":True
                    },
                labels={
                    "PrixMcarré":"Prix du m²"
                })
    fig.update_layout(autosize=True)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

def carteDensite():
    # Dictionnaire associant chaque quartier à son aire
    neighbourhoodArea = dict()
    for cartier in quartierGeo["features"]:
        neighbourhoodArea[cartier["properties"]["neighbourhood"]] = round(area(cartier["geometry"]))
    neighbourhoodArea["Reuilly"] += -9950000 #J'enlève la superficie du bois de Vincenne qui fait partie du 12eme
    #Nombre de logement par quartier:
    mapData = pd.DataFrame(listing.groupby(["neighbourhood_cleansed"])["neighbourhood_cleansed"].count().reset_index(name="#logements"))
    mapData["superficie"] = mapData["neighbourhood_cleansed"].apply(lambda x: neighbourhoodArea[x])
    #nombre de logement par hectares par quartier
    mapData["nb/hectares"] = round(mapData["#logements"]/(mapData["superficie"]/10000), 2)
    mapData = mapData.rename(columns={"neighbourhood_cleansed" : 'neighbourhood'})
    mapData["Arrondissement"] = mapData["neighbourhood"].apply(lambda x: nomToNumero[x])
    fig = px.choropleth_mapbox(mapData, geojson=quartierGeo, color='nb/hectares',
              locations='neighbourhood', featureidkey="properties.neighbourhood",
              mapbox_style="carto-positron", center={"lat":48.86, "lon": 2.35}, zoom=11,
              opacity=0.8, color_continuous_scale='reds',
              labels={
                  "#logements":"Nombre de logements",
                  "nb/hectares":"Nombre de logements par hectares"
                  },
              hover_data={
                  "neighbourhood":False,
                  "Arrondissement":True,
                  "nb/hectares":True,
                  "#logements":True
              })
    fig.update_layout(autosize=True)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

def cartePrix():
    mapData = listing.loc[listing["accommodates"]==2].groupby(["neighbourhood_cleansed"])["price"].mean().reset_index(name="Prix moyen")
    mapData["Prix moyen"] = round(mapData["Prix moyen"],2)
    mapData = mapData.rename(columns={"neighbourhood_cleansed":"neighbourhood"})
    mapData["Arrondissement"] = mapData["neighbourhood"].apply(lambda x: nomToNumero[x])
    fig = px.choropleth_mapbox(mapData, geojson=quartierGeo, color='Prix moyen',
                locations='neighbourhood', featureidkey="properties.neighbourhood",
                mapbox_style="carto-positron", center={"lat":48.86, "lon": 2.35}, zoom=11,
                opacity=0.6, color_continuous_scale='greens',
                hover_data={
                    "neighbourhood":False,
                    "Arrondissement":True,
                })
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(autosize=True)
    return fig

def carteScore():
    mapData = listing.groupby("neighbourhood_cleansed").mean("review_scores_rating")["review_scores_rating"].reset_index(name="Evaluation")
    mapData["Arrondissement"] = mapData["neighbourhood_cleansed"].apply(lambda x: nomToNumero[x])
    mapData["Evaluation"] = round(mapData["Evaluation"],2)
    fig = px.choropleth_mapbox(mapData, geojson=quartierGeo, color='Evaluation',
                locations='neighbourhood_cleansed', featureidkey="properties.neighbourhood",
                mapbox_style="carto-positron", center={"lat":48.86, "lon": 2.35}, zoom=11,
                opacity=0.6, color_continuous_scale='blues',
                hover_data={
                    "neighbourhood_cleansed":False,
                    "Arrondissement":True,
                })
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(autosize=True)
    return fig

def carteValue():
    mapData = listing.groupby("neighbourhood_cleansed").mean("review_scores_value")["review_scores_value"].reset_index(name="Evaluation")
    mapData["Arrondissement"] = mapData["neighbourhood_cleansed"].apply(lambda x: nomToNumero[x])
    mapData["Evaluation"] = round(mapData["Evaluation"],2)
    fig = px.choropleth_mapbox(mapData, geojson=quartierGeo, color='Evaluation',
                locations='neighbourhood_cleansed', featureidkey="properties.neighbourhood",
                mapbox_style="carto-positron", center={"lat":48.86, "lon": 2.35}, zoom=11,
                opacity=0.6, color_continuous_scale='oranges',
                hover_data={
                    "neighbourhood_cleansed":False,
                    "Arrondissement":True,
                })
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(autosize=True)
    return fig

In [ ]:
partie1 = dbc.Card(
    dbc.CardBody(
        [
            html.Div(
                [
                    dbc.Label("Sélection de la carte A"),
                    dcc.Dropdown(
                        options=[
                            {'label':'Prix moyen', 'value':'cartePrix'},
                            {'label':'Densité de logements', 'value':'carteDensite'},
                            {'label':'Prix du mètre carré', 'value':'cartePrixAuMetreCarre'},
                            {'label':'Note générale', 'value':'carteScore'},
                            {'label':'Note qualité-prix', 'value':'carteValue'}
                        ],
                        value = 'cartePrix',
                        id="DD graph1",
                        clearable=False
                        ),
                    html.H4("Nom du graphique 1", id = "Titre1")
                ]
            ),
            dcc.Graph(
                id="graph1",
                figure=cartePrix()
            )
        ]
    ),
    id="carte1"
)

partie2 = dbc.Card(
    dbc.CardBody(
        [
            html.Div(
                [
                    dbc.Label("Sélection de la carte B:"),
                    dcc.Dropdown(
                        options=[
                            {'label':'Prix moyen', 'value':'cartePrix'},
                            {'label':'Densité de logements', 'value':'carteDensite'},
                            {'label':'Prix du mètre carré', 'value':'cartePrixAuMetreCarre'},
                            {'label':'Note générale', 'value':'carteScore'},
                            {'label':'Note qualité-prix', 'value':'carteValue'}
                        ],
                        value = 'carteDensite',
                        id="DD graph2",
                        clearable=False
                        ),
                    html.H4(id='Titre2')
                ]
            ),
            dcc.Graph(
                id="graph2",
                figure=carteDensite()
            )
        ]
    ),
    id="carte2"
)

cards1 = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(partie1),
                dbc.Col(partie2)
            ],
            className="mb-6"
        )
    ]
)

@app.callback(
    Output('graph2', 'figure'),
    Input('DD graph2', 'value')
)
def update_graph2(choix):
    return eval(f"{choix}()")

@app.callback(
    Output('Titre2', 'children'),
    Input('DD graph2', 'value')
)
def update_graph_title2(choix):
    if choix == 'cartePrix': return "Prix moyen pour 2 personnes"
    if choix == 'carteDensite': return "Nombre de logements par hectare"
    if choix == 'cartePrixAuMetreCarre': return "Prix du m²"
    if choix == 'carteScore': return "Note générale du logement sur 5"
    if choix == 'carteValue': return "Note rapport qualité-prix sur 5"
    
@app.callback(
    Output('graph1', 'figure'),
    Input('DD graph1', 'value')
)
def update_graph1(choix):
    return eval(f"{choix}()")

@app.callback(
    Output('Titre1', 'children'),
    Input('DD graph1', 'value')
)
def update_graph_title1(choix):
    if choix == 'cartePrix': return "Prix moyen pour 2 personnes"
    if choix == 'carteDensite': return "Nombre de logements par hectare"
    if choix == 'cartePrixAuMetreCarre': return "Prix du m²"
    if choix == 'carteScore': return "Note générale du logement sur 5"
    if choix == 'carteValue': return "Note rapport qualité-prix sur 5"

app.layout = cards1
app.run_server(port='8888')

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

def prixParChambre():
    colors = ["#89CFF0"]*5
    colors[2]="#DC143C"
    listing.loc[listing["bedrooms"]==0, "bedrooms"] = 0.999
    listing["prixParChambre"] = listing["price"]/listing["bedrooms"]
    listing.loc[listing["bedrooms"]==0.999, "bedrooms"] = 0
    pPC = listing.groupby("bedrooms")["prixParChambre"].mean().reset_index(name="prixParChambre").rename(columns={"bedrooms":"chambres"})
    pPC["prixParChambre"] = pPC["prixParChambre"].apply(lambda x : round(x, 2))
    pPC["chambres"] = pPC["chambres"].apply(lambda x : round(x))
    pPC["Prix/chambre"] = pPC['prixParChambre'].apply(lambda x: f"{round(x)}€")
    pPC = pPC.loc[pPC["chambres"] <= 4]
    fig = px.bar(pPC, x='chambres', y='prixParChambre', text='Prix/chambre',
                 hover_data={'prixParChambre':False},
                 labels={"prixParChambre":"Prix/Chambre"})
    fig.update_traces(marker_color=colors, marker_line_color='rgb(0,0,0)',
                    marker_line_width=2, opacity=1)
    fig.update_layout(title={
        "text":"Prendre un AirBNB à plusieurs est-il rentable ?",
        "x":0.5,
        "font":{
            "size":20
        }
    })
    fig
    return fig

def typeLogement():
    Logements = listing.groupby(["room_type"])["room_type"].count()\
                .reset_index(name="#logements")\
                .sort_values(by=['#logements'],ascending=False)
    Logements["room_type"] = ["Logement entier", "Chambre perso", "Chambre d'hotel","Dortoir"]
    fig = px.pie(Logements, values="#logements", names="room_type", 
                 color_discrete_sequence=px.colors.qualitative.Safe)
    fig.update_layout(title={
        "text":"Type de logements:",
        "x":0.5,
        "font":{
            "size":20
        }
    })
    return fig


histoPrix = dbc.Card(children=[dcc.Graph(id="graph3",figure=prixParChambre())])
pieType = dbc.Card(children=[dcc.Graph(id="graph4", figure=typeLogement())])

cards2 = dbc.Card([
    #titlerow,
    dbc.Row([
        dbc.Col(histoPrix, width=7),
        dbc.Col(pieType, width=5)
    ],
    align="center")
],
body=True)

app.layout = cards2
app.run_server(mode='inline', port='8888')

In [7]:
repartition.head()

,price,neighbourhood_cleansed,bedrooms
0,150,Observatoire,0.0
1,146,Hôtel-de-Ville,0.0
2,110,Entrepôt,0.0
3,140,Hôtel-de-Ville,0.0
4,180,Opéra,0.0


In [15]:
repartition.loc[repartition['bedrooms'] == 0].count()

price                     67073
neighbourhood_cleansed    67073
bedrooms                  67073
dtype: int64

In [16]:
listingRaw["bedrooms"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
74324   NaN
74325   NaN
74326   NaN
74327   NaN
74328   NaN
Name: bedrooms, Length: 74329, dtype: float64

In [8]:
def prixPlot(chambres, arrond):
    prix = repartition[['price', 'neighbourhood_cleansed', 'bedrooms']]
    if chambres != 'Tout':
        prix = prix.loc[(prix['bedrooms'] == int(chambres))]
    if arrond != 'Tout':
        prix = prix.loc[(prix['neighbourhood_cleansed'] == numeroToNom[arrond])]
    #considérons que 93% des valeurs sinon distribution trop grande, graphique moche
    low, high = np.percentile(prix['price'], 0), np.percentile(prix['price'], 93) 
    prix = prix.loc[(prix['price'] < high) & (prix['price'] >= low)]['price']
    prix = prix.rename("Prix en € pour une nuit")
    fig = px.histogram(prix, x="Prix en € pour une nuit", 
                       nbins=30, 
                       color_discrete_sequence=['#800020'],
                       template="plotly_white"
                       )
    fig.update_layout(title={
        "text":"Répartition des prix:",
        "x":0.5,
        "font":{
            "size":20
        }
    })
    return fig

In [10]:
prixPlot('2', 'Tout')

In [9]:
#Répartition des prix

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

prixPlotCallback = dbc.Card(
    children=[
        html.Label("Choisir l'arrondissement:"),
        dcc.Dropdown(['Tout']+list(nomToNumero.values()),
                     value='Tout',
                     id='dropdown_graph4',
                     clearable=False),
        html.Br(),
        html.Label('Nombre de chambres:'),
        dcc.RadioItems(["Tout","1","2","3"], value='Tout', id='radio_graph4')
], body=True)

cards3 = html.Div(children=[
    dbc.Row([
        dbc.Col([dbc.Card(children=[dcc.Graph(id="graph4")])], 
            width=9),
        dbc.Col(prixPlotCallback, width=3)
        ],
    align="start")
    ])

@app.callback(
    Output('graph4', 'figure'),
    Input('radio_graph4', 'value'),
    Input('dropdown_graph4', 'value'))
def prixPlot(chambres, arrond):
    prix = repartition[['price', 'neighbourhood_cleansed', 'bedrooms']]
    if chambres != 'Tout':
        prix = prix.loc[(prix['bedrooms'] == int(chambres))]
    if arrond != 'Tout':
        prix = prix.loc[(prix['neighbourhood_cleansed'] == numeroToNom[arrond])]
    #considérons que 93% des valeurs sinon distribution trop grande, graphique moche
    low, high = np.percentile(prix['price'], 0), np.percentile(prix['price'], 93) 
    prix = prix.loc[(prix['price'] < high) & (prix['price'] >= low)]['price']
    prix = prix.rename("Prix en € pour une nuit")
    fig = px.histogram(prix, x="Prix en € pour une nuit", 
                       nbins=30, 
                       color_discrete_sequence=['indianred'],
                       template="plotly_white"
                       )
    fig.update_layout(title={
        "text":"Répartition des prix:",
        "x":0.5,
        "font":{
            "size":20
        }
    })
    return fig

app.layout = dbc.Container(
    children = [cards3],
    fluid = True,
    className="dbc",
    style={"padding":"0px"}
)
app.run_server(mode='inline', port='8888')

/home/sarintt/anaconda3/envs/myenv/lib/python3.10/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



TypeError: 'NoneType' object cannot be interpreted as an integer